In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

In [2]:
linear_train_df = pd.read_csv('combined_data\linear_train_data.csv')
linear_val_df = pd.read_csv('combined_data\linear_val_data.csv')
linear_test_df = pd.read_csv('combined_data\linear_test_data.csv')
non_linear_train_df = pd.read_csv('combined_data\\non_linear_train_data.csv')
non_linear_val_df = pd.read_csv('combined_data\\non_linear_val_data.csv')
non_linear_test_df = pd.read_csv('combined_data\\non_linear_test_data.csv')
overlapping_train_df = pd.read_csv('combined_data\overlapping_train_data.csv')
overlapping_val_df = pd.read_csv('combined_data\overlapping_val_data.csv')
overlapping_test_df = pd.read_csv('combined_data\overlapping_test_data.csv')

In [3]:
def knn_1nn_classification(train_data, train_labels, test_data, test_labels):

    classifier = KNeighborsClassifier(n_neighbors=1)    
    classifier.fit(train_data, train_labels)
    predicted_labels = classifier.predict(test_data)
    accuracy = accuracy_score(test_labels, predicted_labels)
    
    return predicted_labels, accuracy

In [4]:
# For linearly separable data
linear_train_data = linear_train_df[['X', 'Y']]
linear_train_labels = linear_train_df['Label']

linear_test_data = linear_test_df[['X', 'Y']]
linear_test_labels = linear_test_df['Label']

linear_predicted_labels, linear_accuracy = knn_1nn_classification(linear_train_data, linear_train_labels, 
                                                                  linear_test_data, linear_test_labels)
print("Linearly Separable Data Accuracy:", linear_accuracy)

Linearly Separable Data Accuracy: 1.0


In [5]:
# For non linearly separable data
non_linear_train_data = non_linear_train_df[['X', 'Y']]
non_linear_train_labels = non_linear_train_df['Label']

non_linear_test_data = non_linear_test_df[['X', 'Y']]
non_linear_test_labels = non_linear_test_df['Label']

non_linear_predicted_labels, non_linear_accuracy = knn_1nn_classification(non_linear_train_data, non_linear_train_labels, 
                                                                  non_linear_test_data, non_linear_test_labels)

print("Non Linearly Separable Data Accuracy:", non_linear_accuracy)

Non Linearly Separable Data Accuracy: 1.0


In [6]:
# For overlapping data
overlapping_train_data = overlapping_train_df[['X', 'Y']]
overlapping_train_labels = overlapping_train_df['Label']

overlapping_test_data = overlapping_test_df[['X', 'Y']]
overlapping_test_labels = overlapping_test_df['Label']

overlapping_predicted_labels, overlapping_accuracy = knn_1nn_classification(overlapping_train_data, overlapping_train_labels, 
                                                                  overlapping_test_data, overlapping_test_labels)
print("Overlapping Data Accuracy:", overlapping_accuracy)

Overlapping Data Accuracy: 0.85
